In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syste

In [3]:
from datasets import load_dataset

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
dataset=load_dataset("sumitaryal/nepali_grammatical_error_correction")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/451 [00:00<?, ?B/s]

train-00000-of-00007.parquet:   0%|          | 0.00/217M [00:00<?, ?B/s]

train-00001-of-00007.parquet:   0%|          | 0.00/217M [00:00<?, ?B/s]

train-00002-of-00007.parquet:   0%|          | 0.00/216M [00:00<?, ?B/s]

...

train-00005-of-00007.parquet:   0%|          | 0.00/217M [00:00<?, ?B/s]

train-00006-of-00007.parquet:   0%|          | 0.00/217M [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/79.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7723971 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/406525 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['incorrect_sentence', 'correct_sentence'],
        num_rows: 7723971
    })
    valid: Dataset({
        features: ['incorrect_sentence', 'correct_sentence'],
        num_rows: 406525
    })
})

In [19]:
train_dataset=dataset['train'].select(range(5000))
test_dataset=dataset['valid'].select(range(2000))

In [8]:
from transformers import AutoTokenizer, TFAutoModelForMaskedLM


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [9]:

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = TFAutoModelForMaskedLM.from_pretrained("google/muril-base-cased")

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at google/muril-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [20]:
train_dataset

Dataset({
    features: ['incorrect_sentence', 'correct_sentence'],
    num_rows: 5000
})

In [21]:
def tokenize_dataset(datas):
  inputs=tokenizer(datas['incorrect_sentence'],padding="max_length",max_length=70,truncation=True,return_tensors='tf')
  targets=tokenizer(datas['correct_sentence'],padding="max_length",max_length=70,truncation=True,return_tensors='tf')
  input_ids=inputs.input_ids
  attention_mask=inputs.attention_mask
  labels=targets.input_ids

  return {
      'input_ids':input_ids,
      'attention_mask':attention_mask,
      'labels':labels
  }


In [22]:
train_dataset=train_dataset.map(tokenize_dataset,batched=True)
test_dataset=train_dataset.map(tokenize_dataset,batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [23]:
import tensorflow as tf
from tensorflow import keras

In [24]:
train_dataset

Dataset({
    features: ['incorrect_sentence', 'correct_sentence', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [25]:
Train_dataset=train_dataset.remove_columns(['incorrect_sentence', 'correct_sentence'])
Test_dataset=test_dataset.remove_columns(['incorrect_sentence', 'correct_sentence'])

In [26]:
Train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [27]:
def convert_to_tensorflow(data):
  input_ids=tf.convert_to_tensor(data['input_ids'])
  attention_mask=tf.convert_to_tensor(data['attention_mask'])
  labels=tf.convert_to_tensor(data['labels'])
  dataset=tf.data.Dataset.from_tensor_slices(({
      'input_ids':input_ids,
      'attention_mask':attention_mask,

  },labels))

  return dataset.batch(10)

In [28]:
train_dataset=convert_to_tensorflow(Train_dataset)
test_dataset=convert_to_tensorflow(Test_dataset)

In [29]:
for i in train_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(10, 70), dtype=int32, numpy=
array([[   104,    442, 146463,  15575, 193485,   1397,   4639,  21897,
         10948,    437,  73131,  67336,   8255,   1815,  24971,   2742,
          8431,  68624,   1815, 186876,  50865,  28690,    437,    492,
           105,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0],
       [   104,  11937,   2190,  21702, 149230,   1920,  27141, 181829,
          1227,  58991,    492,    105,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0, 

In [30]:
import tensorflow as tf
from tensorflow import keras

In [31]:
from transformers import AdamWeightDecay

In [32]:
tf.config.run_functions_eagerly(True)

In [33]:
model.compile(
    optimizer=AdamWeightDecay(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

model.summary()

Model: "tf_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  236965632 
                                                                 
 mlm___cls (TFBertMLMHead)   multiple                  152700581 
                                                                 
Total params: 237755045 (906.96 MB)
Trainable params: 237755045 (906.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(train_dataset,validation_data=test_dataset,epochs=5)

Epoch 1/5
  1/500 [..............................] - ETA: 4:53:56 - loss: 15.1720 - accuracy: 0.1829

In [ ]:
sentence="नेपाल को निकासी व्यापारमा ऊनी गलैंचाले बृद्धि भयको छ।"

inputs=tokenizer([sentence],padding=True,max_length=156,truncation=True,return_tensors='tf')
print(inputs)
output=model(**inputs)
print(output)

In [ ]:
def into_output():
  input_ids=tf.convert_to_tensor(inputs['input_ids'])
  attention_mask=tf.convert_to_tensor(inputs['attention_mask'])
  final_input={
      'input_ids':input_ids,
      'attention_mask':attention_mask
  }
  final_output=model(final_input)
  predicted_ids = np.argmax(final_output.logits,axis=-1)
  print(tokenizer.decode(predicted_ids[0],skip_special_tokens=True))

into_output()


